In [76]:
import os
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt


## Preprocessing

### Loading data

In [77]:
TOP_STOCKS_OPENING_CLOSING_PRICES = '../../../data/top_stocks_opening_closing_prices_data/'
top_stocks_opening_closing_prices_files = os.listdir(TOP_STOCKS_OPENING_CLOSING_PRICES)
top_stocks_opening_closing_prices_dfs = []
for filename in top_stocks_opening_closing_prices_files:
    if filename.endswith(".csv"):
        print(filename)
        df = pd.read_csv(TOP_STOCKS_OPENING_CLOSING_PRICES + filename)
        top_stocks_opening_closing_prices_dfs.append(df)
        continue
    else:
        continue
top_stocks_opening_closing_prices_dfs = pd.concat(top_stocks_opening_closing_prices_dfs, axis=0, ignore_index=True)
top_stocks_opening_closing_prices_dfs

top_stocks_opening_closing_prices_05-25-2023.csv
top_stocks_opening_closing_prices_05-26-2023.csv
top_stocks_opening_closing_prices_05-29-2023.csv
top_stocks_opening_closing_prices_05-30-2023.csv
top_stocks_opening_closing_prices_05-31-2023.csv
top_stocks_opening_closing_prices_06-01-2023.csv
top_stocks_opening_closing_prices_06-02-2023.csv


,stock,opening_price,closing_price
0,NVDA,385.230,379.80
1,AI,30.500,28.41
2,AMD,117.305,120.35
3,PLTR,13.180,12.84
4,TSLA,186.540,184.47
...,...,...,...
65,AVGO,790.640,812.00
66,SOFI,7.110,7.03
67,DG,162.200,166.12
68,META,272.660,272.61


In [78]:
top_stocks_info_df = pd.DataFrame()
TOP_STOCKS_INFO_PATH = '../../../data/top_stocks_info_data/'
top_stocks_info_files = os.listdir(TOP_STOCKS_INFO_PATH)
top_stocks_info_dfs = []
for filename in top_stocks_info_files:
    if filename.endswith(".csv"):
        print(filename)
        df = pd.read_csv(TOP_STOCKS_INFO_PATH + filename)
        top_stocks_info_dfs.append(df)
        continue
    else:
        continue
top_stocks_info_df = pd.concat(top_stocks_info_dfs, axis=0, ignore_index=True)
top_stocks_info_df

top_stocks_info_05-25-2023.csv
top_stocks_info_05-26-2023.csv
top_stocks_info_05-29-2023.csv
top_stocks_info_05-30-2023.csv
top_stocks_info_05-31-2023.csv
top_stocks_info_06-01-2023.csv
top_stocks_info_06-02-2023.csv


,timestamp,rank,ticker,name,mentions,mentioning_users,upvotes,sentiment,rank_24h_ago,mentions_24h_ago,beta,epsTTM,peTTM,roeTTM,dividendYieldIndicatedAnnual,totalDebt/totalEquityQuarterly,revenueGrowthTTMYoy
0,05-25-2023,1,NVDA,NVIDIA,3216,1370.0,30860,57.0,3,275,1.770654,1.7415,173.5333,18.66000,0.052394,0.4956,0.2200
1,05-25-2023,2,AI,C3 AI,1363,889.0,8328,68.0,1,327,2.707838,-2.4261,NaN,-22.98162,NaN,0.0000,14.6000
2,05-25-2023,3,AMD,AMD,403,259.0,2718,64.0,7,92,1.812139,0.2412,131.9033,2.40000,NaN,0.0451,22.2000
3,05-25-2023,6,PLTR,Palantir,145,69.0,354,69.0,4,141,1.682084,-0.1246,NaN,-7.15660,NaN,0.0000,20.5000
4,05-25-2023,7,TSLA,Tesla,128,104.0,861,64.0,5,101,1.753238,3.3967,49.9490,27.90000,NaN,0.0557,38.3400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,06-02-2023,8,AVGO,Broadcom,113,74.0,274,58.0,26,31,1.109033,30.2074,26.3232,58.26000,2.329261,1.6852,20.7400
66,06-02-2023,9,SOFI,SoFi,108,62.0,403,72.0,5,218,1.943151,-0.2665,NaN,-1.40908,NaN,1.1056,162.6023
67,06-02-2023,10,DG,Dollar General,90,65.0,2156,77.0,119,12,0.359083,10.6825,18.2371,40.63000,1.458050,1.2648,10.5900
68,06-02-2023,11,META,Meta Platforms (Facebook),79,59.0,291,56.0,25,28,1.392399,7.8343,31.6414,17.14000,NaN,0.0960,-1.9400


### Joining prices and info datsets

The opening_and_closing_prices data and top_stocks_info data would have to be concatanated. 

In [79]:
training_data = pd.concat([top_stocks_opening_closing_prices_dfs, top_stocks_info_df], axis=1)

### Computing target labels

In [80]:
training_data['label'] = (training_data['opening_price'] -
                          training_data['closing_price']).apply(lambda x: 1 if x > 0 else 0)

In [81]:
training_data['label'].value_counts()

label
0    44
1    26
Name: count, dtype: int64

### Renaming columns

In [82]:
training_data.rename(
    columns={'dividendYieldIndicatedAnnual': 'dividend_yield_annual',
             'epsTTM': 'earnings_per_share_ttm',
             'peTTM': 'price_to_equity_ttm',
             'roeTTM': 'return_on_equity_ttm',
             'totalDebt/totalEquityQuarterly': 'total_debt_to_equity_quarterly',
             'revenueGrowthTTMYoy': 'revenue_growth_ttm_yoy'
             }, inplace=True)

### Creating new columns

In [83]:
training_data['dividend_exists'] = training_data['dividend_yield_annual'].apply(lambda x: 1 if x > 0 else 0)

In [84]:
training_data['change_in_rank'] = (
    (training_data['rank'] - training_data['rank_24h_ago'])/training_data['rank_24h_ago']
)

In [85]:
training_data['change_in_mentions'] = (
    (training_data['mentions'] - training_data['mentions_24h_ago'])/training_data['mentions_24h_ago']
)

In [86]:
training_data.isnull().sum()

stock                              0
opening_price                      0
closing_price                      0
timestamp                          0
rank                               0
ticker                             0
name                               0
mentions                           0
mentioning_users                   0
upvotes                            0
sentiment                          0
rank_24h_ago                       0
mentions_24h_ago                   0
beta                               0
earnings_per_share_ttm             0
price_to_equity_ttm               19
return_on_equity_ttm               0
dividend_yield_annual             40
total_debt_to_equity_quarterly     0
revenue_growth_ttm_yoy             0
label                              0
dividend_exists                    0
change_in_rank                     0
change_in_mentions                 0
dtype: int64

### Dropping columns with duplication

In [87]:
training_data.drop(
    columns = ['stock'],
    inplace=True
)

### Imputing values in columns

In [88]:
training_data['dividend_yield_annual'].replace(to_replace=np.nan, value=0, inplace=True)
training_data['price_to_equity_ttm'].replace(to_replace=np.nan, value=0, inplace=True)

### Saving training and testing data

In [89]:
training_data.describe()

,opening_price,closing_price,rank,mentions,mentioning_users,upvotes,sentiment,rank_24h_ago,mentions_24h_ago,beta,earnings_per_share_ttm,price_to_equity_ttm,return_on_equity_ttm,dividend_yield_annual,total_debt_to_equity_quarterly,revenue_growth_ttm_yoy,label,dividend_exists,change_in_rank,change_in_mentions
count,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000
mean,191.463786,192.616429,6.971429,279.200000,175.457143,2618.742857,66.600000,25.071429,218.942857,1.672243,4.192324,85.904277,23.786502,0.582838,0.883074,21.607131,0.371429,0.428571,-0.185002,3.492647
std,186.616217,186.551978,3.615472,503.059267,267.851012,5447.304686,11.160749,47.984394,471.981942,0.653953,8.608851,219.614012,45.258641,1.301282,3.228772,38.810489,0.486675,0.498445,0.498431,9.359938
min,5.240000,5.410000,1.000000,20.000000,19.000000,67.000000,38.000000,1.000000,1.000000,0.359083,-14.138900,0.000000,-32.949290,0.000000,0.000000,-27.400000,0.000000,0.000000,-0.962712,-0.625000
25%,32.970000,33.385000,4.000000,58.250000,43.500000,291.750000,58.250000,4.250000,20.500000,1.161613,-0.124600,0.000000,-1.409080,0.000000,0.045100,2.140000,0.000000,0.000000,-0.578341,-0.102564
50%,144.345000,140.875000,7.500000,100.500000,68.500000,558.500000,67.000000,8.000000,50.000000,1.708573,1.832850,29.405950,19.770000,0.000000,0.289850,14.600000,0.000000,0.000000,-0.190909,0.727221
75%,310.595000,312.592500,10.000000,202.500000,122.250000,1599.000000,74.750000,24.750000,143.250000,1.812139,5.885700,112.635425,28.365000,0.555801,0.969500,22.200000,1.000000,1.000000,0.083333,2.677390
max,835.120000,812.000000,13.000000,3216.000000,1370.000000,30860.000000,100.000000,295.000000,3050.000000,4.772926,39.367900,1783.525900,165.720000,8.620690,27.021900,162.602300,1.000000,1.000000,1.000000,57.000000


In [90]:
training_data

,opening_price,closing_price,timestamp,rank,ticker,name,mentions,mentioning_users,upvotes,sentiment,...,earnings_per_share_ttm,price_to_equity_ttm,return_on_equity_ttm,dividend_yield_annual,total_debt_to_equity_quarterly,revenue_growth_ttm_yoy,label,dividend_exists,change_in_rank,change_in_mentions
0,385.230,379.80,05-25-2023,1,NVDA,NVIDIA,3216,1370.0,30860,57.0,...,1.7415,173.5333,18.66000,0.052394,0.4956,0.2200,1,1,-0.666667,10.694545
1,30.500,28.41,05-25-2023,2,AI,C3 AI,1363,889.0,8328,68.0,...,-2.4261,0.0000,-22.98162,0.000000,0.0000,14.6000,1,0,1.000000,3.168196
2,117.305,120.35,05-25-2023,3,AMD,AMD,403,259.0,2718,64.0,...,0.2412,131.9033,2.40000,0.000000,0.0451,22.2000,0,0,-0.571429,3.380435
3,13.180,12.84,05-25-2023,6,PLTR,Palantir,145,69.0,354,69.0,...,-0.1246,0.0000,-7.15660,0.000000,0.0000,20.5000,1,0,0.500000,0.028369
4,186.540,184.47,05-25-2023,7,TSLA,Tesla,128,104.0,861,64.0,...,3.3967,49.9490,27.90000,0.000000,0.0557,38.3400,1,0,0.400000,0.267327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,790.640,812.00,06-02-2023,8,AVGO,Broadcom,113,74.0,274,58.0,...,30.2074,26.3232,58.26000,2.329261,1.6852,20.7400,0,1,-0.692308,2.645161
66,7.110,7.03,06-02-2023,9,SOFI,SoFi,108,62.0,403,72.0,...,-0.2665,0.0000,-1.40908,0.000000,1.1056,162.6023,1,0,0.800000,-0.504587
67,162.200,166.12,06-02-2023,10,DG,Dollar General,90,65.0,2156,77.0,...,10.6825,18.2371,40.63000,1.458050,1.2648,10.5900,0,1,-0.915966,6.500000
68,272.660,272.61,06-02-2023,11,META,Meta Platforms (Facebook),79,59.0,291,56.0,...,7.8343,31.6414,17.14000,0.000000,0.0960,-1.9400,1,0,-0.560000,1.821429


In [91]:
training_data.columns

Index(['opening_price', 'closing_price', 'timestamp', 'rank', 'ticker', 'name',
       'mentions', 'mentioning_users', 'upvotes', 'sentiment', 'rank_24h_ago',
       'mentions_24h_ago', 'beta', 'earnings_per_share_ttm',
       'price_to_equity_ttm', 'return_on_equity_ttm', 'dividend_yield_annual',
       'total_debt_to_equity_quarterly', 'revenue_growth_ttm_yoy', 'label',
       'dividend_exists', 'change_in_rank', 'change_in_mentions'],
      dtype='object')

In [92]:
training_data.to_csv('../../../data/train_test_data.csv', index=False)